In [22]:
#imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [23]:
#dados
train=pd.read_csv('/content/drive/MyDrive/titanic/train.csv')
test=pd.read_csv('/content/drive/MyDrive/titanic/test.csv')

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [25]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [26]:
#tratando os dados
#separando por titulo
for df in [train, test]:
    df["Title"] = df["Name"].apply(lambda x: x.split(',')[1].split('.')[0].strip() if ',' in x and '.' in x.split(',')[1] else '')
    df["Title"] = df["Title"].replace(["Don", "Rev", "Dr", "Major", "Lady", "Sir", "Col", "Capt", "the Countess", "Jonkheer"], "Rare")
    df["Title"] = df["Title"].replace(["Mlle", "Ms"], "Miss")
    df["Title"] = df["Title"].replace(["Mme"], "Mrs")

In [27]:
target='Survived'
x_train,x_test,y_train,y_test=train_test_split(train.drop(target,axis=1),train[target],test_size=.3,random_state=0)

df_train=pd.concat([x_train,y_train],axis=1)
df_test=pd.concat([x_test,y_test],axis=1)

for i in [df_train,df_test,test]:
    i.drop(['Name','Ticket'],axis=1,inplace=True)

In [28]:
categorical_features=[i for i in df_train.columns if df_train[i].dtype=='O']
numerical_features=[i for i in df_train.columns if df_train[i].dtype!='O']
print('categorical_features :\n',categorical_features,'\nnumerical_features :\n',numerical_features)

categorical_features :
 ['Sex', 'Cabin', 'Embarked', 'Title'] 
numerical_features :
 ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']


In [29]:
#irq removendo outliers
for feature in numerical_features:
    if 0 in df_train[feature].unique():
        pass
    else:
        q1=df_train[feature].quantile(0.25)
        q3=df_train[feature].quantile(0.75)
        iqr=q3-q1
        lower_limit=q1-1.5*iqr
        upper_limit=q3+1.5*iqr
        df_train=df_train[(df_train[feature]<upper_limit) & (df_train[feature]>lower_limit)]

In [30]:
#tratando dados nulos
#preenchendo valores ausentes em 'Age' com a mediana
df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
df_test['Age'].fillna(df_test['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)

#preenchendo valores ausentes em 'Embarked' com o valor mais frequente
df_train['Embarked'].fillna(df_train['Embarked'].mode()[0], inplace=True)
df_test['Embarked'].fillna(df_test['Embarked'].mode()[0], inplace=True)
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)

#preenchendo valores ausentes em 'Fare' com a mediana
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

for df in [df_train, df_test, test]:
    df['Cabin'].fillna('Unknown', inplace=True)
    df['Cabin'] = df['Cabin'].astype(str)

print(df_train.isnull().sum())
print(df_test.isnull().sum())
print(test.isnull().sum())

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
Title          0
Survived       0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
Title          0
Survived       0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
Title          0
dtype: int64


In [31]:
# Separando as features novamente após o tratamento de dados
X_train = df_train.drop('Survived', axis=1)
y_train = df_train['Survived']
X_test = df_test.drop('Survived', axis=1)
y_test = df_test['Survived']

In [32]:
# Redefinindo as listas de features numéricas e categóricas sem a coluna 'Survived'
# Isso é crucial pois X_train e X_test não possuem mais a coluna 'Survived'
numerical_features = [i for i in X_train.columns if X_train[i].dtype != 'O']
categorical_features = [i for i in X_train.columns if X_train[i].dtype == 'O']

In [33]:
# Criando o pipeline de pré-processamento
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [34]:
# Criando o modelo SVM
model = SVC()

In [35]:
# Criando o pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

In [36]:
# Treinando o modelo
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['PassengerId', 'Pclass',
                                                   'Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Cabin', 'Embarked',
                                                   'Title'])])),
                ('classifier', SVC())])

In [37]:
# Fazendo as predições no conjunto de teste
y_pred = pipeline.predict(X_test)

In [45]:
# Avaliando o modelo
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity= tn/(tn+fp)

accuracy=(tp+tn)/(tp+tn+fp+fn)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=2*(precision*recall)/(precision + recall)

print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Especificidade",specificity)
print("Recall:", recall)
print("F1-Score:", f1_score)


              precision    recall  f1-score   support

           0       0.84      0.87      0.86       168
           1       0.77      0.73      0.75       100

    accuracy                           0.82       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.82      0.82      0.82       268

[[146  22]
 [ 27  73]]
Acurácia: 0.8171641791044776
Precisão: 0.7684210526315789
Especificidade 0.8690476190476191
Recall: 0.73
F1-Score: 0.7487179487179486


In [40]:
predictions = pipeline.predict(test)

In [42]:
# Criando um DataFrame com os resultados
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions
})

# Salvando o DataFrame em um arquivo CSV
submission.to_csv('titanicSVM.csv', index=False)